# explore-streams_view
For more information refer to: [IBM Streams Python support](https://streamsxtopology.readthedocs.io/en/stable/index.html)

# Views

Streams provides a view into the executing application in order that you may monitor the current processing. The 
accompanying application composed and submitted by [streaming_analytics_lfe_pipeline](streaming_analytics_lfe_pipeline.jupyter-py36.ipynb) has a number of these views. 
This notebook illustrates accessing the views and rendering the streams of a running application. 

Views were designed as a monitoring and exploration tool. Views are a shared resource, 
an viewing an operator processing significant amounts of data can impact other views. 
Effecting other views includes losing tuples and lagging data, for this reason views 
should be relegated to exploration, debugging, and monitoring.

The goal of this notebook is to demonstrate views. Views used in 
conjunction with standard Jupyter facilities for live monitoring of 
of the data processing.

**This project contains Sample Materials, provided under license.  
Licensed Materials - Property of IBM.  
© Copyright IBM Corp. 2019. All Rights Reserved.  
US Government Users Restricted Rights - Use, duplication or disclosure restricted by GSA ADP Schedule Contract with IBM Corp.**


In [ ]:
import json
import sys
import urllib3
import collections
import time

from IPython.display import display, clear_output
import matplotlib.pyplot as plt
%matplotlib inline

from streamsx.topology import context
import streamsx.rest as rest
if '../scripts' not in sys.path:
    sys.path.insert(0, '../scripts')
from streams_render import list_jobs
from streams_render import display_views
from streams_render import find_jobs

from streams_render import display_view_stop


## Initialization

### Connect to your instance

In [ ]:
#Use the Connect to Instance toolbar button and insert code for config
#e.g.
from icpd_core import icpd_util
cfg=icpd_util.get_service_instance_details(name='zen-sample-icp1-blitz-env')

### Configure the environment.

In [ ]:
# Get the Streams instance setup set some values in order to run more smoothly in notebook.
cfg[context.ConfigParams.SSL_VERIFY] = False
instance = rest.Instance.of_service(cfg)
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
# Create a dictionary of Views..
jobs = find_jobs(instance)
VIEWS = dict({})
for job in jobs:
    VIEWS = {**VIEWS, **{view.name:view for view in job.get_views()} }


### Check the status of the running jobs..
The running application is composed of a number of jobs. The following command lists the application and their state.  

In [ ]:
## Verify Streams instance is running!
list_jobs(instance, cancel=False)

### Monitor the data as it moves through the application.

Display radio buttons of views of the application. Selecting 
a view will display it's current data. As data moves through the application the 
messages are consolidated: many 'real_time_events' flow in, a number of events
(100+ per customer) are necessary to generate a 'new_score', only scores that 
pass a significance tests are rendered in the 'significant_events' view. The m

In [ ]:
display_views(instance)

## Monitor the data entering the system, view : 'real_time_events'

The 'real_time_event' view allows monitoring of data as it arrives. 

The follow displays a table of arriving events. 

In [ ]:
VIEWS['real_time_events'].display(duration=30)

### Another perspective on 'real_time_events'.

Looking at table can be overwhelming. An api is provided to fetch the view data in order to render it in an more informative manner. 
Below, using the same view's data we render a pie chart. 


In [ ]:
# Change at your discretion.
wait_time_between_renderings = 5
number_of_iterations = 3
for x in range(number_of_iterations):
    elems = [json.loads(item.data['jsonString']) for item in VIEWS['real_time_events'].get_view_items()]
    cnt = collections.Counter()
    events = [elem[2] for elem in elems]
    for event in events:
        cnt[event] += 1
    labels=[key for key in cnt.keys()]
    values=[cnt[ele] for ele in cnt.keys()]
    plt.pie(values, labels=values)
    plt.legend(labels, loc=0)
    plt.show()
    time.sleep(wait_time_between_renderings)
    clear_output(wait=True)

## Monitor  the scores that are generated, view : 'new_scores'
For each event that arrives two scores are attempted for the associated customer.

The 'new_scores' view is monitoring the newly generated scores derived from the 'real_time_events'. 
Using the display() you can see scores are flowing. 

In [ ]:
VIEWS['new_scores'].display(duration=30)

### Looking at a bigger picture. 

In order to generate the score, the models require a number of events. The Streams 
application provides the events to the model, the model determines 
if it has sufficient events to generate a score. In this example two models are
being used, each of which determines it's events requirements. 

Consolidating the events into a score makes the data more dense. Using the same view 
data and the power of notebooks we can show the current scoring distribution of the
two models being scored. 

In [ ]:
# change at your discretion 
wait_time_between_renderings = 5
number_of_iterations = 3
for x in range(number_of_iterations):
    elems = [json.loads(item.data['jsonString']) for item in VIEWS['new_scores'].get_view_items()]
    purchase = [elem['lfe_home_purchase']['probabilities'][0][1] for elem in elems if 'probabilities' in elem['lfe_home_purchase']]
    relocation = [elem['lfe_relocation']['probabilities'][0][1] for elem in elems if 'probabilities' in elem['lfe_relocation']]
    #plots the histogram
    fig, ax1 = plt.subplots()
    ax1.hist([relocation,purchase],color=['b','g'])
    ax1.set_ylabel("Count")
    plt.legend(["lfe_home_purchase", "lfe_relocation"], loc='center left', bbox_to_anchor=(1, .5))
    plt.show()
    time.sleep(wait_time_between_renderings)
    clear_output(wait=True)

## Looking 'significant events'
In the context of this application a significant event may result in a notification (email, text)
being sent to a responsible party. The number of sifnificatant events are configured to be low to avoid 
'alarm fatigue'.


In [ ]:
VIEWS['significant_events'].display(duration=30)

## More of significance.

The following display the complete significant events.

In [ ]:
[json.loads(item.data['jsonString']) for item in VIEWS['significant_events'].get_view_items()]